# Step 1: Create an amplitude model

{mod}`tensorwaves` requires you to first formulate an amplitude model that you want to fit to your data set. The {mod}`expertsystem` helps you to construct such a model.

This notebook briefly illustrates how to create such an amplitude model with the {mod}`expertsystem` and how to write it to a recipe file that can be understood by {mod}`tensorwaves`. For more control, have a look at {doc}`the usage guides of the PWA Expert System <expertsystem:usage>`.

In this example, we use the helicity formalism, but you can also use `formalism_type="canonical-helicity"`. As you can see, we analyze the decay $J/\psi \to \pi^0\pi^0\gamma$ here.

```{admonition} Simplified model: $J/\psi \to f_0\gamma$
---
class: dropdown
---
As {doc}`3_perform_fit` serves to illustrate usage only, we make the amplitude model here a bit simpler by not allowing $\omega$ resonances (which are narrow and therefore hard to fit). For this reason, we can also limit the {class}`~expertsystem.reaction.default_settings.InteractionTypes` to {attr}`~expertsystem.reaction.default_settings.InteractionTypes.STRONG`.
```

In [ ]:
import expertsystem as es

result = es.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
    allowed_interaction_types="strong and EM",
)

As a small goodie, you can use [`graphviz`](https://pypi.org/project/graphviz) to {doc}`visualize the found graphs <expertsystem:usage/visualization>`:

In [ ]:
from graphviz import Source

graphs = result.collapse_graphs()
dot = es.io.convert_to_dot(graphs)
Source(dot)

Next we convert the {attr}`~expertsystem.reaction.Result.transitions` into an {class}`~expertsystem.amplitude.model.AmplitudeModel`. This can be done with the {func}`~expertsystem.generate_amplitudes` method.

In [ ]:
model = es.generate_amplitudes(result)
list(model.parameters)

Note that we have to specify the dynamics for the resonances. We choose to use {class}`~expertsystem.amplitude.model.RelativisticBreitWigner` for all resonances:

In [ ]:
for name in result.get_intermediate_particles().names:
    model.dynamics.set_breit_wigner(name)
{name: type(dyn) for name, dyn in model.dynamics.items()}

Finally, we can write the {class}`~expertsystem.amplitude.model.AmplitudeModel`, using the {func}`~expertsystem.io.write` function:

In [ ]:
es.io.write(model, "amplitude_model_helicity.yml")

Cool, that's it! We now have a recipe for an amplitude model with which to {doc}`generate data <2_generate_data>` and {doc}`perform a fit <3_perform_fit>`! In the next steps, we will use use this {class}`~expertsystem.amplitude.model.AmplitudeModel` as a fit model template for {mod}`tensorwaves`.